# Model to Predict Elo Customer Loyalty

_Note! If you want to commit any changes to this document, please strip all output (Cell > Current Outputs > Clear, or set up [nbstripout](https://github.com/kynan/nbstripout) as a git filter) from this notebook before doing so. Thanks!_


## Import Libraries

Next we import the Python libraries we'll need. If any of these are missing for you, you can install them with e.g. `pip3 install pandas` on the command line.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

## Load Data

Load the data into Pandas data frames and look at their structure.

First thing we'll do with the training data is split it into a train and validation set. (The given test set is what we'll later make our predictions on and upload, but only after we are fully satisfied with our model.)

**Make sure to run `make processdata` (which takes a very long time, but only needs to be done once) before running the code in this notebook!**

In [ ]:
hist_trans_df = pd.read_csv('data/processed/historical_transactions.csv')
merch_trans_df = pd.read_csv('data/processed/new_merchant_transactions_with_merchants.csv')
train_and_validation_df = pd.read_csv('data/unzipped/train.csv',
                                      index_col='card_id',
                                      parse_dates=['first_active_month'])
test_df = pd.read_csv('data/unzipped/test.csv',
                      index_col='card_id',
                      parse_dates=['first_active_month'])

In [ ]:
hist_trans_df.head()

In [ ]:
merch_trans_df.head()

In [ ]:
train_and_validation_df.head()

## Create Features

Next we want to combine and shape all of our raw data to create useful features in the train (and validation and test) data set.

In [ ]:
from fastai import *
from fastai.tabular import *
from fastai.metrics import *
from feature_engineering import *

In [ ]:
add_datepart(train_and_validation_df, 'first_active_month')

In [ ]:
train_and_validation_df.drop(['first_active_monthDay', 'first_active_monthDayofweek',
                              'first_active_monthDayofyear', 'first_active_monthIs_month_end',
                              'first_active_monthIs_month_start', 'first_active_monthIs_quarter_end',
                              'first_active_monthIs_year_end'], axis=1, inplace=True)

In [ ]:
add_datepart(test_df, 'first_active_month')

In [ ]:
test_df.drop(['first_active_monthDay', 'first_active_monthDayofweek', 'first_active_monthDayofyear',
              'first_active_monthIs_month_end', 'first_active_monthIs_month_start',
              'first_active_monthIs_quarter_end', 'first_active_monthIs_year_end'], axis=1, inplace=True)

### Aggregate Transaction Data

Next we'll use the functions defined in `feature_engineering.py` to aggregate the historical transactions for each card into single values for that card, for instance the mean of all purchase amounts, &c.

_Note: these functions can take quite a long time to complete._

In [ ]:
hist_trans_df.columns

### Historical Transactions

In [ ]:
aggs = {
    'purchase_amount': ['sum', 'mean', 'min', 'max', 'std'],
    'installments': ['sum', 'mean', 'min', 'max', 'std'],
    'month_lag': ['mean', 'min', 'max'],
    'merchant_id': ['nunique'],
    'state_id': ['nunique'],
    'city_id': ['nunique'],
}

In [ ]:
# Here are the aggregators we only want to use for the `historical_transactions` data.
hist_trans_aggs = {
    'merchant_category_id': ['nunique'],
    'subsector_id': ['nunique'],
    'elapsed_since_last_purchase': ['sum', 'mean', 'min', 'max', 'std'],
    'elapsed_since_last_merch_purchase': ['sum', 'mean', 'min', 'max', 'std'],
}

In [ ]:
add_aggregated_numerical_fields(train_and_validation_df, hist_trans_df, aggregators={**aggs, **hist_trans_aggs})

For the categorical fields, we can't aggregate by taking the mean or sum values, so let's count the occurences of each possible categorical value instead. _(Iow, for a category that can be either YES or NO, we count the number of YESes and the number of NOs and use those values.)_

In [ ]:
add_aggregated_categorical_fields(train_and_validation_df,
                                  hist_trans_df,
                                  column_names=['authorized_flag', 'category_1', 'category_2', 'category_3'])

In [ ]:
add_top_categories(train_and_validation_df,
                   hist_trans_df,
                   column_names=['authorized_flag', 'category_1', 'subsector_id', 'city_id', 'state_id',
                                 'purchase_Year', 'purchase_Month', 'purchase_Week', 'purchase_Day',
                                 'purchase_Dayofweek'])

### New Merchant Transactions

Now lets do the same aggregating for the `new_merchants_transactions` data.

In [ ]:
merch_trans_df.columns

In [ ]:
# Here are the aggregators we only want to use for the `new_merchants_transactions` data.
merch_trans_aggs = {
    'category_1_transaction': ['nunique'],
    'category_2': ['nunique'],
    'category_3': ['nunique'],
    'category_4': ['nunique'],
    'merchant_category_id_transaction': ['nunique'],
    'merchant_category_id_merchant': ['nunique'],
    'merchant_group_id': ['nunique'],
    'subsector_id_merchant': ['nunique'],
    'category_1_merchant': ['nunique'],
    'state_id': ['nunique'],
    'elapsed_since_last_purchase': ['sum', 'mean', 'min', 'max', 'std'],
    'numerical_1': ['sum', 'mean', 'min', 'max', 'std'],
    'numerical_2': ['sum', 'mean', 'min', 'max', 'std'],
    'avg_sales_lag3': ['sum', 'mean', 'min', 'max', 'std'],
    'avg_purchases_lag3': ['sum', 'mean', 'min', 'max', 'std'],
    'active_months_lag3': ['sum', 'mean', 'min', 'max', 'std'],
    'avg_sales_lag6': ['sum', 'mean', 'min', 'max', 'std'],
    'avg_purchases_lag6': ['sum', 'mean', 'min', 'max', 'std'],
    'active_months_lag6': ['sum', 'mean', 'min', 'max', 'std'],
    'avg_sales_lag12': ['sum', 'mean', 'min', 'max', 'std'],
    'avg_purchases_lag12': ['sum', 'mean', 'min', 'max', 'std'],
    'active_months_lag12': ['sum', 'mean', 'min', 'max', 'std'],
}

In [ ]:
add_aggregated_numerical_fields(train_and_validation_df, merch_trans_df, aggregators={**aggs, **merch_trans_aggs},
                                prefix='merch_')

For some reason this fails for the `merch_trans_df` data frame, so let's skip these for now.

In [ ]:
#add_aggregated_categorical_fields(train_and_validation_df,
#                                  merch_trans_df,
#                                  column_names=['category_1_transaction', 'merchant_category_id_transaction',
#                                                'state_id', 'purchase_Year', 'month_lag'],
#                                  prefix='merch_')

In [ ]:
#add_top_categories(train_and_validation_df,
#                   merch_trans_df,
#                   column_names=['category_1_transaction', 'merchant_category_id_transaction',
#                                 'state_id', 'purchase_Year', 'month_lag'],
#                   prefix='merch_')

In [ ]:
train_and_validation_df.head()

### Repeat for Test Set

This takes a long time, too, of course.

In [ ]:
add_aggregated_numerical_fields(test_df, hist_trans_df, aggregators={**aggs, **hist_trans_aggs})
add_aggregated_categorical_fields(test_df,
                                  hist_trans_df,
                                  column_names=['authorized_flag', 'category_1', 'category_2', 'category_3'])
add_top_categories(test_df,
                   hist_trans_df,
                   column_names=['authorized_flag', 'category_1', 'subsector_id', 'city_id', 'state_id',
                                 'purchase_Year', 'purchase_Month', 'purchase_Week', 'purchase_Day',
                                 'purchase_Dayofweek'])

In [ ]:
add_aggregated_numerical_fields(test_df, merch_trans_df, aggregators={**aggs, **merch_trans_aggs},
                                prefix='merch_')

## Split Into Train and Validation Sets

Split our data into a train test (80%) and a validation set (20%).

In [ ]:
from sklearn.model_selection import train_test_split
train_df, validate_df = train_test_split(train_and_validation_df, test_size=0.2, random_state=238923)

In [ ]:
train_df.shape

In [ ]:
validate_df.shape

In [ ]:
train_df.head()

## Remove Outliers

We shouldn't actually ever do this manually, except for experimental purposes. Spoiler: the outliers have a large impact on the final performance of our model.

In [ ]:
# train_df = train_df[train_df.target > -25]

## A Quick Look at Correlations

In [ ]:
train_df.corr().target.sort_values(ascending=False)

## Set Up Model

We'll use the fastai tabular regressor here, which is built for exactly this problem.

### Create Data Bunch

A fastai DataBunch more or less contains the data that we'll feed to our model.

First, as the data bunch takes one data frame containing both the test and validation samples, we need to get the indices for our validation samples.

Then we tell the model which of the columns are categorical features, which are continuous features, and also which of the columns contains the target (the value we want to predict).

In [ ]:
valid_idx = range(len(train_df), len(train_df) + len(validate_df)); valid_idx

Let's have a look at which columns we have. We will need to tell fastai which ones are categorical and which ones are continuous.

In [ ]:
for c in train_df.columns: print(c)

In [ ]:
category_names = ['feature_1',
                  'feature_2',
                  'feature_3',
                  'authorized_flag_top',
                  'category_1_top',
                  'subsector_id_top',
                  'city_id_top',
                  'state_id_top',
                  'purchase_Year_top',
                  'purchase_Month_top',
                  'purchase_Week_top',
                  'purchase_Day_top',
                  'purchase_Dayofweek_top']
continuous_names = ['first_active_monthYear',
                    'first_active_monthMonth',
                    'first_active_monthWeek',
                    'first_active_monthIs_quarter_start',
                    'first_active_monthIs_year_start',
                    'first_active_monthElapsed',
                    'purchase_amount_sum',
                    'purchase_amount_mean',
                    'purchase_amount_min',
                    'purchase_amount_max',
                    'purchase_amount_std',
                    'installments_sum',
                    'installments_mean',
                    'installments_min',
                    'installments_max',
                    'installments_std',
                    'month_lag_mean',
                    'month_lag_min',
                    'month_lag_max',
                    'merchant_id_nunique',
                    'state_id_nunique',
                    'city_id_nunique',
                    'merchant_category_id_nunique',
                    'subsector_id_nunique',
                    'elapsed_since_last_purchase_sum',
                    'elapsed_since_last_purchase_mean',
                    'elapsed_since_last_purchase_min',
                    'elapsed_since_last_purchase_max',
                    'elapsed_since_last_purchase_std',
                    'elapsed_since_last_merch_purchase_sum',
                    'elapsed_since_last_merch_purchase_mean',
                    'elapsed_since_last_merch_purchase_min',
                    'elapsed_since_last_merch_purchase_max',
                    'elapsed_since_last_merch_purchase_std',
                    'authorized_flag_Y_ratio',
                    'category_1_N_ratio',
                    'category_2_1.0_ratio',
                    'category_2_3.0_ratio',
                    'category_2_4.0_ratio',
                    'category_2_2.0_ratio',
                    'category_2_5.0_ratio',
                    'category_3_A_ratio',
                    'category_3_B_ratio',
                    'category_3_C_ratio',
                    'merch_purchase_amount_sum',
                    'merch_purchase_amount_mean',
                    'merch_purchase_amount_min',
                    'merch_purchase_amount_max',
                    'merch_purchase_amount_std',
                    'merch_installments_sum',
                    'merch_installments_mean',
                    'merch_installments_min',
                    'merch_installments_max',
                    'merch_installments_std',
                    'merch_month_lag_mean',
                    'merch_month_lag_min',
                    'merch_month_lag_max',
                    'merch_merchant_id_nunique',
                    'merch_state_id_nunique',
                    'merch_city_id_nunique',
                    'merch_category_1_transaction_nunique',
                    'merch_category_2_nunique',
                    'merch_category_3_nunique',
                    'merch_category_4_nunique',
                    'merch_merchant_category_id_transaction_nunique',
                    'merch_merchant_category_id_merchant_nunique',
                    'merch_merchant_group_id_nunique',
                    'merch_subsector_id_merchant_nunique',
                    'merch_category_1_merchant_nunique',
                    'merch_elapsed_since_last_purchase_sum',
                    'merch_elapsed_since_last_purchase_mean',
                    'merch_elapsed_since_last_purchase_min',
                    'merch_elapsed_since_last_purchase_max',
                    'merch_elapsed_since_last_purchase_std',
                    'merch_numerical_1_sum',
                    'merch_numerical_1_mean',
                    'merch_numerical_1_min',
                    'merch_numerical_1_max',
                    'merch_numerical_1_std',
                    'merch_numerical_2_sum',
                    'merch_numerical_2_mean',
                    'merch_numerical_2_min',
                    'merch_numerical_2_max',
                    'merch_numerical_2_std',
                    'merch_avg_sales_lag3_sum',
                    'merch_avg_sales_lag3_mean',
                    'merch_avg_sales_lag3_min',
                    'merch_avg_sales_lag3_max',
                    'merch_avg_sales_lag3_std',
                    'merch_avg_purchases_lag3_sum',
                    'merch_avg_purchases_lag3_mean',
                    'merch_avg_purchases_lag3_min',
                    'merch_avg_purchases_lag3_max',
                    'merch_avg_purchases_lag3_std',
                    'merch_active_months_lag3_sum',
                    'merch_active_months_lag3_mean',
                    'merch_active_months_lag3_min',
                    'merch_active_months_lag3_max',
                    'merch_active_months_lag3_std',
                    'merch_avg_sales_lag6_sum',
                    'merch_avg_sales_lag6_mean',
                    'merch_avg_sales_lag6_min',
                    'merch_avg_sales_lag6_max',
                    'merch_avg_sales_lag6_std',
                    'merch_avg_purchases_lag6_sum',
                    'merch_avg_purchases_lag6_mean',
                    'merch_avg_purchases_lag6_min',
                    'merch_avg_purchases_lag6_max',
                    'merch_avg_purchases_lag6_std',
                    'merch_active_months_lag6_sum',
                    'merch_active_months_lag6_mean',
                    'merch_active_months_lag6_min',
                    'merch_active_months_lag6_max',
                    'merch_active_months_lag6_std',
                    'merch_avg_sales_lag12_sum',
                    'merch_avg_sales_lag12_mean',
                    'merch_avg_sales_lag12_min',
                    'merch_avg_sales_lag12_max',
                    'merch_avg_sales_lag12_std',
                    'merch_avg_purchases_lag12_sum',
                    'merch_avg_purchases_lag12_mean',
                    'merch_avg_purchases_lag12_min',
                    'merch_avg_purchases_lag12_max',
                    'merch_avg_purchases_lag12_std',
                    'merch_active_months_lag12_sum',
                    'merch_active_months_lag12_mean',
                    'merch_active_months_lag12_min',
                    'merch_active_months_lag12_max',
                    'merch_active_months_lag12_std',]
dep_var = 'target'

Since we picked our validation samples randomly from the initial data set, and since fastai requires us to give the indices of the validation samples in a data frame containing both the training and validation samples, we just concatenate them together with training samples first and the validation samples at the end.

In [ ]:
df = pd.concat([train_df, validate_df]).reset_index()[category_names + continuous_names + [dep_var]]

In [ ]:
data = (TabularList.from_df(df,
                            path='data/unzipped',
                            cat_names=category_names,
                            cont_names=continuous_names,
                            procs=[FillMissing, Categorify, Normalize])
                .split_by_idx(valid_idx)
                .label_from_df(cols=dep_var, label_cls=FloatList)
                .databunch())

Let's have a look at a random batch of data to see how it looks after the processing done by the fastai library.

In [ ]:
data.show_batch()

### Create Learner

This is what we actually use to train the model and make predictions.

First we decide how large we want to make the embeddings of our categorical features (the number of category options divided by 2 is a good heuristic, apparently).

Then we tell the model the range within which we expect all predictions to fall (internally the model uses a sigmoid function, so in order for us, in practice, to actually get predictions near the expected maximum value, we set the upper bound to be a little higher than the expected maximum).

The competition uses root mean squared error to evaluate the entries, so we'll use that, too.

In [ ]:
min_y = np.min(train_df['target']) * 1.2
max_y = np.max(train_df['target']) * 1.2
y_range = torch.tensor([min_y, max_y], device=defaults.device); y_range

In [ ]:
np.min(train_df['target']), np.max(train_df['target'])

In [ ]:
learn = tabular_learner(data,
                        layers=[400, 200],
                        ps=[2e-1, 1e-1],
                        emb_drop=0.04,
                        y_range=y_range,
                        metrics=rmse)

In [ ]:
learn.model

### Figure Out Learning Rate

To figure out which learning rate to use, we use fastai's learning rate finder.

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

### Train Model

Finally we train the model, with weight decay to encourage the model to use fewer features, and then show some results.

In [ ]:
learn.fit_one_cycle(3, 1e-3, wd=0.7)

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.recorder.show_results()

## Make Predictions

Now that we have trained our model, lets make some predictions to see whether or not our metrics lie to us.

In [ ]:
predictions, targets = [x.numpy().flatten() for x in learn.get_preds(DatasetType.Valid)]
prediction_df = pd.DataFrame({'prediction': predictions, 'target': targets})

In [ ]:
(np.amin(predictions), np.amax(predictions))

In [ ]:
prediction_df.head()

In [ ]:
prediction_df.tail()

### Calculate RMSE On Validation Set

Get the root mean squared error for the validation set only. This value we can compare against the public leaderboard on Kaggle, more or less.

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
sqrt(mean_squared_error(prediction_df.target, prediction_df.prediction))

## Make Submission Predictions

Finally, we need to run our model against the test set that is used by the competition's organizers to evaluate the competitors. We save the result to a `submission.csv` file which we'll then upload to Kaggle.

_Note: we should only do this at the very end, when we are happy with our hyperparameters. Otherwise, if we change our model based on our results on the public leaderboard, we risk overfitting our model to the 30% of samples used for the public leaderboard, and will fail to generalize for the remaining 70% of samples._

In [ ]:
out_df = test_df.copy(); out_df.head()

The test set has one row with some missing values (which we don't have in the training set), so let's use the most commonly occuring ones for that row.

In [ ]:
out_df.fillna(value={'first_active_monthYear': '2017.0',
                     'first_active_monthMonth': '12.0',
                     'first_active_monthWeek': '44.0'}, inplace=True)

In [ ]:
# Warning -- this takes quite a long time.
from tqdm import tqdm_notebook as tqdm
targets = []
for _, row in tqdm(out_df.iterrows()):
    targets.append(learn.predict(row)[2].numpy().flatten()[0])
out_df['target'] = pd.Series(targets)

In [ ]:
out_df['target'].to_csv('submission.csv.zip', header=['target'], index_label='card_id', compression='zip')